In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import glob
import re

# Read In Hits 

In [ ]:
ds6_int = pd.read_table('../../deadsea/deadn6.htab', sep='\t', lineterminator='\n', skiprows=0)
ds6_int

In [ ]:
for indy,row in ds6_int.iterrows():
    if row['hmm _oord_from'] < 25 or row['hmm_coord_to'] == 471:
        continue
    else:
        ds6_int.drop(index=indy, inplace=True)
ds6_int

In [ ]:
ds6_int.reset_index(inplace=True, drop=True)
ds6_int

## Intein Hit Defragmenter

In [ ]:
# iterrows and refer to previous rows *****
# defragments intein hits in htab
for indy, row in ds6_int.iterrows():
    if row['hmm _oord_from'] == 1:
        continue
    if row['hmm_coord_to'] == 471:
        if row['target_name'] == ds6_int.loc[(int(indy) - 1), 'target_name']:
            if row['hmm _oord_from'] > (ds6_int.loc[(int(indy) - 1), 'hmm_coord_to']):
                ds6_int.loc[(int(indy) - 1),
                            'env_coord_to'] = row['env_coord_to']

In [ ]:
#get rid of C-terms after adding coords
index_delete = []
for indy, row in ds6_int.iterrows():
    if row['hmm _oord_from'] == 1 and row['hmm_coord_to'] == 471:
        continue
    if row['env_coord_to'] == ds6_int.loc[(int(indy) -1), 'env_coord_to'] and row['target_name'] == ds6_int.loc[(int(indy) -1), 'target_name']:
        index_delete.append(indy)

In [ ]:
ds6_int.drop(index=index_delete, inplace=True)
ds6_int.reset_index(inplace=True, drop=True)
ds6_int

# Sequence Indexer
## Intein Seq Get (AA)

In [ ]:
inteins = {}
for index, row in ds6_int.iterrows():
    if row['target_name'] in inteins.keys():
        inteins.setdefault(row['target_name'], []).append(row['env_coord_to'])        
        
    else:
        inteins[row['target_name']] = [ row['env_coord_to']]

In [ ]:
#read in fasta
faa_fasta = {}
with open("../../deadsea/deadn6.faa.eol") as faa_file:
    for line in faa_file:
        line = line.strip()
        if not line:
            continue
        if line.startswith(">"):
            active_sequence_name = line[1:].split(' ')[0]
            if active_sequence_name not in faa_fasta.keys():
                faa_fasta[active_sequence_name] = ""
            continue
        sequence = line
        faa_fasta[active_sequence_name] = sequence

faa_fasta

In [ ]:
#all intein containing genes
for prot in inteins.keys():
    print('>' + prot + '\n' + faa_fasta[prot], file=open('intein_genes.fasta', 'a'))

In [ ]:
# all invidiual intein seqs
for indy, row in ds6_int.iterrows():
    start_site = (row['env_coord_from'] - 1)
    stop_site = (row['env_coord_to'])
    print(">" + row['target_name'] + '_' + str(row['dom#']) + '\n' +
          faa_fasta[row['target_name']][start_site:stop_site])  # , file=open('all_int.fasta', 'a') )

## Extein Seq Get (AA)

In [ ]:
# formatting the sites
ext_sites = {}
for index, row in ds6_int.iterrows():
    if row['target_name'] in inteins.keys():
        ext_sites.setdefault(row['target_name'], []).append(
            row['env_coord_from'])
        ext_sites[row['target_name']].append(row['env_coord_to'])
    else:
        ext_sites[row['target_name']] = [
            row['env_coord_from'], row['env_coord_to']]

ext_sites
# extsites to change for extein grabbbinh

### Fix Site Index Slicers

In [ ]:
fix_ext_sites = {}
for target, vals in ext_sites.items():
    dom_num = 0
    expression = ''
    while dom_num < len(vals):
        if dom_num == 0:
            expression = expression + ":" + str(vals[dom_num]) + ","
            dom_num += 1
        if dom_num % 2 != 0:
            expression = expression + (str(vals[dom_num] - 1)) + ":"
            dom_num += 1
        else:
            expression = expression + str(vals[dom_num]) + ','
            dom_num += 1
    fix_ext_sites[target] = expression.split(',')
fix_ext_sites

### Concat Extein Seqs (AA) 

In [ ]:
# get concatenated extein sequences from fix_ext_sites
extein_seqs = {}
for target, vals in fix_ext_sites.items():
    sequence = ''
    for pair in vals:
        if pair[0] == ':':
            first_end = int(pair.split(':')[1])
            sequence = sequence + faa_fasta[target][:first_end]
        else:
            start = int(pair.split(':')[0])

            if pair[-1] == ':':
                sequence = sequence + faa_fasta[target][start:]
            else:
                end = int(pair.split(':')[1])
                sequence = sequence + faa_fasta[target][start:end]

    extein_seqs[target] = sequence

In [ ]:
for target,vals in extein_seqs.items():
    print('>' + target + '\n' + vals )#, file=open('ext_seqs.fasta', 'a'))

## Seq Get (NT)

In [ ]:
ffn_fasta = {}
with open("../../deadsea/deadn6.ffn.eol") as ffn_file:
    for line in ffn_file:
        line = line.strip()
        if not line:
            continue
        if line.startswith(">"):
            active_sequence_name = line[1:].split(' ')[0]
            if active_sequence_name not in ffn_fasta.keys():
                ffn_fasta[active_sequence_name] = ""
            continue
        sequence = line
        ffn_fasta[active_sequence_name] = sequence

ffn_fasta

In [ ]:
# get concatenated extein sequences from fix_ext_sites
nt_extein_seqs = {}
for target, vals in fix_ext_sites.items():
    sequence = ''
    for pair in vals:
        if pair[0] == ':':
            first_end = int(pair.split(':')[1])
            sequence = sequence + ffn_fasta[target][:(first_end*3)]
        else:
            start = int(pair.split(':')[0])

            if pair[-1] == ':':
                sequence = sequence + ffn_fasta[target][(start*3):]
            else:
                end = int(pair.split(':')[1])
                sequence = sequence + ffn_fasta[target][(3*start):(3*end)]

    nt_extein_seqs[target] = sequence

In [ ]:
for target, seqs in nt_extein_seqs.items():
    print('>' + target + '_ext_full\n' +
          ffn_fasta[target], file=open('../../deadsea/seq_out/nt_full_ext_seqs.fasta', 'a'))

In [ ]:
# all invidiual NT intein seqs
for indy, row in ds6_int.iterrows():
    start_site = (row['env_coord_from'] - 1)
    stop_site = (row['env_coord_to'])
    print(">" + row['target_name'] + '_' + str(row['dom#']) + '\n' + ffn_fasta[row['target_name']]
          [(start_site*3):(stop_site*3)], file=open('../../deadsea/seq_out/nt_int.fasta', 'a'))

# Homing Endonuclease

In [ ]:
#read in blocked file, into a list.uniq(), if present in list and inteins dict then its a full
full_inteins = []
for line in open('../../deadsea/seq_out/hensearch/all_int_low.blocked').readlines():
    line = line.split()
    full_inteins.append(line[0])
full_uniqs=  list(set(full_inteins))

In [ ]:
ds6_int['int_size'] = ''
for indy, row in ds6_int.iterrows():
    if (row['target_name'] + '_' + str(row['dom#'])) in full_uniqs:
        ds6_int.loc[indy, 'int_size'] = 'full'
    else:
        ds6_int.loc[indy, 'int_size'] = 'mini'
ds6_int        

## Value Calculation

In [ ]:
multi_hen_dict = {}
for indy, row in ds6_int.iterrows():
    if row['target_name'] in multi_hen_dict.keys():
        multi_hen_dict.setdefault(
            row['target_name'], []).append(row['int_size'])

    else:
        multi_hen_dict[row['target_name']] = [row['int_size']]
multi_hen_dict

In [ ]:
# uri_multi int (later for mini inteins i think)
ds_gene_count = len(inteins)  # number of total genes with 1+ inteins
# multi_hens = 0 #skip but number of hens per multi genes
multi_inteins = 0  # multiple inteins total (count of inteins)
# single intein in gene. if only 1 target_name (count of gene/inteins)
single = 0
# single intein in gene w/HEN. (count of inteins) if only 1 target name and full
single_hen = 0
# genes with multiple inteins (count of genes). if multiple target_names
multiple_genes = 0
# multiple inteins with hen (count of inteins). if multiple target_names, and full
multiple_hen = 0
# multiple_hen/multi_inteins = % of multiple inteins with HEN

for target, vals in multi_hen_dict.items():
    if len(vals) == 1:
        single += 1
        if vals[0] == 'full':
            single_hen += 1
    if len(vals) > 1:

        multiple_genes += 1
        multi_inteins += len(vals)
        for size in vals:
            if size == 'full':
                multiple_hen += 1
# need to change the flat values to be extensible
print('Dead Sea\nGenes with multiple inteins:\t' + str(multiple_genes) + ' (' + str(ds_gene_count) + ')\t' + str(multiple_genes/ds_gene_count) +
      '\nMulti-Inteins with HEN:\t\t' + str(multiple_hen) + '\t\t' + str(multiple_hen/multi_inteins) +
      '\nSingle-Inteins with HEN:\t' + str(single_hen) + '\t\t' + str(single_hen/single) + 
     '\nTotal full inteins:\t\t' + str(single_hen + multiple_hen) + ' (' + str(single + multi_inteins) + ')\t' + str((single_hen+multiple_hen)/(single + multi_inteins)))

# NR mapped

In [ ]:
%pwd

In [ ]:
nr_info = pd.read_csv('../../deadsea/exteinbackground.tab', sep='\t', header=None, usecols=[0, 3])
nr_info.columns = ['target_name', 'title']
nr_info.drop_duplicates(subset='target_name', inplace=True)
ds6_int_anno = ds6_int.merge(nr_info, how='left', left_on='target_name', right_on='target_name')
ds6_int_anno

In [ ]:
ds6_int_anno['organism'] = ds6_int_anno['title'].map(lambda x: re.search(r'\[.+?(?=\])|$', str(x)).group())
ds6_int_anno['gene'] = ds6_int_anno['title'].map(lambda x: re.search(r'^.+?(?=\[)|$', str(x)).group())
ds6_int_anno.drop(['title'], axis=1, inplace=True)
ds6_int_anno['organism'] = ds6_int_anno['organism'].map(lambda x: x.lstrip('['))
ds6_int_anno

In [ ]:
ds6_int_anno.organism.unique()

# Frequency Data

## Chalkboard Tests

In [ ]:
fix_ext_sites['PROKKA_35178'] #35178 has 5 reads over IIS

## Read in SAM
After you convert the bam map file using Samtools view

In [ ]:
#get samtools view "regions"
for target, sites in ext_sites.items():
    site = 0
    anchor = 0
    if len(sites) > 2:
        while site < (len(sites) -1):
        
            if site == 0:
                anchor = anchor +  (sites[site] *3)
                print(target + "_ext:" + str(anchor-3) + '-' + str(anchor+3) )
                site += 2
            if site % 2 == 0 and site != 0:
                anchor = anchor + ((sites[site] - sites[site-1]) *3) 
                print(target + "_ext:"+ str(anchor-3) + '-' + str(anchor+3))
                site +=2
    else:
        anchor = anchor +  (sites[site] *3)
        print(target + "_ext:" + str(anchor-3) + '-' + str(anchor+3) )

In [ ]:
#post region call
iis_sam = pd.read_table('../../deadsea/seq_out/iis/testview.sam',
                        usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13], header=None, sep='\t')
iis_sam.columns = ['read', 'bits', 'target_name',
                   'pos', 'mapq', 'cigar', 'mate_target', 'mpos', 'tlen' ,  'nm', 'as', 'xs']
iis_sam

In [ ]:
iis_counter = {}
iis_lens = {}
number  = 0
for index, row in iis_sam.iterrows():
    if '3D' in row['cigar']:
        
        if row['target_name'][:-4] in iis_counter.keys():
            iis_counter.setdefault(row['target_name'][:-4], []).append(row['cigar'])
        else:
            iis_counter[row['target_name'][:-4]] = [row['cigar']]
for target, vals in iis_counter.items():
    iis_lens[target] = len(vals)
iis_lens
    

# Manual Curation

In [ ]:
ds6_int_man = pd.read_csv('../../deadsea/ds6_int_anno.tab', header=None, sep='\t')
ds6_int_man.columns = ['target_name', 'accession', 'tlen', 'query_name', 'accession.1', 'qlen',
       'full_sequence_E-value', 'full_sequence_score', 'full_sequence_bias',
       'dom#', 'ndom', 'c-Evalue', 'i-Evalue', 'score', 'bias',
       'hmm _oord_from', 'hmm_coord_to', 'ali_coord_from', 'ali_coord_to',
       'env_coord_from', 'env_coord_to', 'acc', 'man_desc', 'description_of_target',
       'int_size', 'species', 'gene']
ds6_int_man

In [ ]:
ds6_int_man = pd.read_csv('../../deadsea/ds6_int_anno.tab', header=None, sep='\t')
ds6_int_man.columns = ['target_name', 'accession', 'tlen', 'query_name', 'accession.1', 'qlen',
       'full_sequence_E-value', 'full_sequence_score', 'full_sequence_bias',
       'dom#', 'ndom', 'c-Evalue', 'i-Evalue', 'score', 'bias',
       'hmm _oord_from', 'hmm_coord_to', 'ali_coord_from', 'ali_coord_to',
       'env_coord_from', 'env_coord_to', 'acc', 'man_desc', 'description_of_target',
       'int_size', 'species', 'gene']
ds6_int_man['allele'] = ''
for index, row in ds6_int_man.iterrows():
    abridged = row['man_desc'].split('_')[0]
    ds6_int_man.loc[index, 'allele'] = abridged
ds6_int_man

In [ ]:

extein_cat_seqs = {}
for index, row in ds6_int_man.iterrows():
    if row['allele'] in extein_cat_seqs.keys():
           extein_cat_seqs.setdefault(row['allele'], []).append(row['target_name'])
    else:
        extein_cat_seqs[row['allele']] = [row['target_name']]
extein_cat_seqs


In [ ]:
uniq_targets ={}
for allele, targets in extein_cat_seqs.items():
    uniq_exts =  list(set(targets))
    uniq_targets[allele] = uniq_exts
for allele, targets in uniq_targets.items():
    for seq in targets:
        print('>' + seq + '\n' + extein_seqs[seq], file=open('../../deadsea/seq_out/' + allele + '.ext', 'a'))

In [ ]:
ds6_int_man['domain'] = ''
ds6_int_man.set_value(146, 'species', 'unclass')
ds6_int_man.set_value(142, 'species', 'unclass')
ds6_int_man.set_value(152, 'species', 'unclass')

for index, row in ds6_int_man.iterrows():
    if 'virus' in row['species'] or 'phage' in row['species']:
        ds6_int_man.set_value(index, 'domain', 'virus')
    elif 'Nealsonbacteria' in row['species'] or 'Sinorhizobium' in row['species']:
        ds6_int_man.set_value(index, 'domain', 'bacteria')
    elif row['species'] == 'unclass':
        ds6_int_man.set_value(index, "domain", 'unclass')
    else:
        ds6_int_man.set_value(index, 'domain', 'archaea')
ds6_int_man

In [ ]:
#typo in manual curation fixed
ds6_int_man.loc[136, 'man_desc'] = 'hypo'

In [ ]:
for index, row in ds6_int_man.iterrows():
    if 'hypo' in row['allele']:
        ds6_int_man.loc[index, 'allele'] = 'hypo'

In [ ]:
ds6_domains = ds6_int_man.groupby('domain').count()
ds6_domain_plotable = dict(ds6_domains.target_name)
ds6_alleles = ds6_int_man.groupby('allele').count()
ds6_alleles_plotable = dict(ds6_alleles.target_name)
ds6_alleles_plotable
ds6_alleles_spec = ds6_int_man.groupby('int_spec').count()
ds6_alleles_spec_plotable = dict(ds6_alleles_spec.target_name)
ds6_alleles_spec_plotable

In [ ]:
ds6_int_man['int_spec'] = ""
for index, row in ds6_int_man.iterrows():
    abridged = row['man_desc'].split(',')[0]
    ds6_int_man.loc[index, 'int_spec'] = abridged
ds6_int_man.int_spec.unique()


# Atlit 22

In [ ]:
at22_int = pd.read_table('../../atlit/at22.htab', sep='\t', lineterminator='\n', skiprows=0)
for indy,row in at22_int.iterrows():
    if row['hmm _oord_from'] < 25 or row['hmm_coord_to'] == 471:
        continue
    else:
        at22_int.drop(index=indy, inplace=True)
at22_int.reset_index(inplace=True, drop=True)

In [ ]:
for indy, row in at22_int.iterrows():
    if row['hmm _oord_from'] == 1:
        continue
    if row['hmm_coord_to'] == 471:
        if row['target_name'] == at22_int.loc[(int(indy) - 1), 'target_name']:
            if row['hmm _oord_from'] > (at22_int.loc[(int(indy) - 1), 'hmm_coord_to']):
                at22_int.loc[(int(indy) - 1),
                            'env_coord_to'] = row['env_coord_to']

In [ ]:
at_index_delete = []
for indy, row in at22_int.iterrows():
    if row['hmm _oord_from'] == 1 and row['hmm_coord_to'] == 471:
        continue
    if indy == 0:
        continue 
    if row['env_coord_to'] == at22_int.loc[(int(indy) - 1), 'env_coord_to'] and row['target_name'] == at22_int.loc[(int(indy) - 1), 'target_name']:
        at_index_delete.append(indy)

In [ ]:
at22_int.drop(index=at_index_delete, inplace=True)
at22_int.reset_index(inplace=True, drop=True)
at22_int

In [ ]:
at_inteins = {}
for index, row in at22_int.iterrows():
    if row['target_name'] in at_inteins.keys():
        at_inteins.setdefault(row['target_name'], []).append(row['env_coord_to'])        
        
    else:
        at_inteins[row['target_name']] = [ row['env_coord_to']]

In [ ]:
#read in fasta
at_faa_fasta = {}
with open("../../atlit/at22.faa.eol") as at_faa_file:
    for line in at_faa_file:
        line = line.strip()
        if not line:
            continue
        if line.startswith(">"):
            active_sequence_name = line[1:].split(' ')[0]
            if active_sequence_name not in at_faa_fasta.keys():
                at_faa_fasta[active_sequence_name] = ""
            continue
        sequence = line
        at_faa_fasta[active_sequence_name] = sequence

at_faa_fasta

In [ ]:
for prot in at_inteins.keys():
    print('>' + prot + '\n' + at_faa_fasta[prot]) # file=open('../../atlit/intein_cds.fasta', 'a'))

In [ ]:
for indy, row in at22_int.iterrows():
    start_site = (row['env_coord_from'] - 1)
    stop_site = (row['env_coord_to'])
    print(">" + row['target_name'] + '_' + str(row['dom#']) + '\n' +
          at_faa_fasta[row['target_name']][start_site:stop_site] )#, file=open('../../atlit/all_int.fasta', 'a') )

In [ ]:
at22_int.loc[at22_int['target_name'] == 'PROKKA_18003']

In [ ]:
# formatting the sites
at_ext_sites = {}
for index, row in at22_int.iterrows():
    if row['target_name'] in at_inteins.keys():
        at_ext_sites.setdefault(row['target_name'], []).append(
            row['env_coord_from'])
        at_ext_sites[row['target_name']].append(row['env_coord_to'])
    else:
        at_ext_sites[row['target_name']] = [
            row['env_coord_from'], row['env_coord_to']]

at_ext_sites

In [ ]:
at_fix_ext_sites = {}
for target, vals in at_ext_sites.items():
    dom_num = 0
    expression = ''
    while dom_num < len(vals):
        if dom_num == 0:
            expression = expression + ":" + str(vals[dom_num]) + ","
            dom_num += 1
        if dom_num % 2 != 0:
            expression = expression + (str(vals[dom_num] - 1)) + ":"
            dom_num += 1
        else:
            expression = expression + str(vals[dom_num]) + ','
            dom_num += 1
    at_fix_ext_sites[target] = expression.split(',')
at_fix_ext_sites

In [ ]:
# get concatenated extein sequences from fix_ext_sites
at_extein_seqs = {}
for target, vals in at_fix_ext_sites.items():
    sequence = ''
    for pair in vals:
        if pair[0] == ':':
            first_end = int(pair.split(':')[1])
            sequence = sequence + at_faa_fasta[target][:first_end]
        else:
            start = int(pair.split(':')[0])

            if pair[-1] == ':':
                sequence = sequence + at_faa_fasta[target][start:]
            else:
                end = int(pair.split(':')[1])
                sequence = sequence + at_faa_fasta[target][start:end]

    at_extein_seqs[target] = sequence

In [ ]:
for target,vals in at_extein_seqs.items():
    print('>' + target + '\n' + vals, file=open('../../atlit/ext_seqs.fasta', 'a'))

In [ ]:
at_ffn_fasta = {}
with open("../../atlit/at22.ffn.eol") as at_ffn_file:
    for line in at_ffn_file:
        line = line.strip()
        if not line:
            continue
        if line.startswith(">"):
            active_sequence_name = line[1:].split(' ')[0]
            if active_sequence_name not in at_ffn_fasta.keys():
                at_ffn_fasta[active_sequence_name] = ""
            continue
        sequence = line
        at_ffn_fasta[active_sequence_name] = sequence

at_ffn_fasta

In [ ]:
# get concatenated extein sequences from fix_ext_sites
at_nt_extein_seqs = {}
for target, vals in at_fix_ext_sites.items():
    sequence = ''
    for pair in vals:
        if pair[0] == ':':
            first_end = int(pair.split(':')[1])
            sequence = sequence + at_ffn_fasta[target][:(first_end*3)]
        else:
            start = int(pair.split(':')[0])

            if pair[-1] == ':':
                sequence = sequence + at_ffn_fasta[target][(start*3):]
            else:
                end = int(pair.split(':')[1])
                sequence = sequence + at_ffn_fasta[target][(3*start):(3*end)]

    at_nt_extein_seqs[target] = sequence
for target, seqs in at_nt_extein_seqs.items():
    print('>' + target + '_ext_full\n' +
          at_ffn_fasta[target], file=open('../../atlit/nt_full_ext_seqs.fasta', 'a'))

In [ ]:
for target, seqs in at_nt_extein_seqs.items():
    print('>' + target + '_ext\n' +
          seqs, file=open('../../atlit/nt_ext_seqs.fasta', 'a'))

In [ ]:
# all invidiual NT intein seqs
for indy, row in at22_int.iterrows():
    start_site = (row['env_coord_from'] - 1)
    stop_site = (row['env_coord_to'])
    print(">" + row['target_name'] + '_' + str(row['dom#']) + '\n' + at_ffn_fasta[row['target_name']]
          [(start_site*3):(stop_site*3)], file=open('../../atlit/nt_int.fasta', 'a'))

In [ ]:
at_full_inteins = []
for line in open('../../atlit/hensearch/at22.blocked').readlines():
    line = line.split()
    at_full_inteins.append(line[0])
at_full_uniqs=  list(set(at_full_inteins))

In [ ]:
at22_int['int_size'] = ''
for indy, row in at22_int.iterrows():
    if (row['target_name'] + '_' + str(row['dom#'])) in at_full_uniqs:
        at22_int.loc[indy, 'int_size'] = 'full'
    else:
        at22_int.loc[indy, 'int_size'] = 'mini'
at22_int        

In [ ]:
at_multi_hen_dict = {}
for indy, row in at22_int.iterrows():
    if row['target_name'] in at_multi_hen_dict.keys():
        at_multi_hen_dict.setdefault(
            row['target_name'], []).append(row['int_size'])

    else:
        at_multi_hen_dict[row['target_name']] = [row['int_size']]


In [ ]:
# uri_multi int (later for mini inteins i think)
at_ds_gene_count = len(at_inteins)  # number of total genes with 1+ inteins
# multi_hens = 0 #skip but number of hens per multi genes
at_multi_inteins = 0  # multiple inteins total (count of inteins)
# single intein in gene. if only 1 target_name (count of gene/inteins)
at_single = 0
# single intein in gene w/HEN. (count of inteins) if only 1 target name and full
at_single_hen = 0
# genes with multiple inteins (count of genes). if multiple target_names
at_multiple_genes = 0
# multiple inteins with hen (count of inteins). if multiple target_names, and full
at_multiple_hen = 0
# multiple_hen/multi_inteins = % of multiple inteins with HEN

for target, vals in at_multi_hen_dict.items():
    if len(vals) == 1:
        at_single += 1
        if vals[0] == 'full':
            at_single_hen += 1
    if len(vals) > 1:

        at_multiple_genes += 1
        at_multi_inteins += len(vals)
        for size in vals:
            if size == 'full':
                at_multiple_hen += 1
# need to change the flat values to be extensible
print('Atlit Coast\nGenes with multiple inteins:\t' + str(at_multiple_genes) + ' (' + str(at_ds_gene_count) + ')\t' + str(at_multiple_genes/at_ds_gene_count) +
      '\nMulti-Inteins with HEN:\t\t' + str(at_multiple_hen) + '\t\t' + str(at_multiple_hen/at_multi_inteins) +
      '\nSingle-Inteins with HEN:\t' + str(at_single_hen) + '\t\t' + str(at_single_hen/at_single) + 
     '\nTotal full inteins:\t\t' + str(at_single_hen + at_multiple_hen) + ' (' + str(at_single + at_multi_inteins) + ')\t' + str((at_single_hen+at_multiple_hen)/(at_single + at_multi_inteins)))

In [ ]:
at_nr_info = pd.read_csv('../../atlit/nr/at_exteinbackground.tab', sep='\t', header=None, usecols=[0, 3])
at_nr_info.columns = ['target_name', 'title']
at_nr_info.drop_duplicates(subset='target_name', inplace=True)
at_int_anno = at22_int.merge(at_nr_info, how='left', left_on='target_name', right_on='target_name')
at_int_anno

In [ ]:
at_int_anno['organism'] = at_int_anno['title'].map(lambda x: re.search(r'\[.+?(?=\])|$', str(x)).group())
at_int_anno['gene'] = at_int_anno['title'].map(lambda x: re.search(r'^.+?(?=\[)|$', str(x)).group())
at_int_anno.drop(['title'], axis=1, inplace=True)
at_int_anno['organism'] = at_int_anno['organism'].map(lambda x: x.lstrip('['))
at_int_anno

## Atlit Frequencies

In [ ]:
for target, sites in at_ext_sites.items():
    site = 0
    anchor = 0
    if len(sites) > 2:
        while site < (len(sites) -1):
        
            if site == 0:
                anchor = anchor +  (sites[site] *3)
                print(target + "_ext:" + str(anchor-3) + '-' + str(anchor+3) )
                site += 2
            if site % 2 == 0 and site != 0:
                anchor = anchor + ((sites[site] - sites[site-1]) *3) 
                print(target + "_ext:"+ str(anchor-3) + '-' + str(anchor+3))
                site +=2
    else:
        anchor = anchor +  (sites[site] *3)
        print(target + "_ext:" + str(anchor-3) + '-' + str(anchor+3) )

In [ ]:
at_iis_sam = pd.read_table('../../atlit/iis/testview_combo.sam',
                        usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13], header=None, sep='\t')
at_iis_sam.columns = ['read', 'bits', 'target_name',
                   'pos', 'mapq', 'cigar', 'mate_target', 'mpos', 'tlen' ,  'nm', 'as', 'xs']
at_iis_sam

In [ ]:
at_iis_counter = {}
at_iis_lens = {}
#number  = 0
for index, row in at_iis_sam.iterrows():
    if '3D' in row['cigar']:
        
        if row['target_name'][:-4] in at_iis_counter.keys():
            at_iis_counter.setdefault(row['target_name'][:-4], []).append(row['cigar'])
        else:
            at_iis_counter[row['target_name'][:-4]] = [row['cigar']]
for target, vals in at_iis_counter.items():
    at_iis_lens[target] = len(vals)
at_iis_lens

#40088 artifact

## Manual Curation

In [ ]:
at_int_man = pd.read_csv('../../atlit/at_int_anno', sep='\t')
at_int_man.columns = ['target_name', 'accession', 'tlen', 'query_name', 'accession.1', 'qlen',
       'full_sequence_E-value', 'full_sequence_score', 'full_sequence_bias',
       'dom#', 'ndom', 'c-Evalue', 'i-Evalue', 'score', 'bias',
       'hmm _oord_from', 'hmm_coord_to', 'ali_coord_from', 'ali_coord_to',
       'env_coord_from', 'env_coord_to', 'acc', 'man_desc', 'description_of_target',
       'int_size', 'species', 'gene']
at_int_man

In [ ]:
at_int_man['allele'] = ''
for index, row in at_int_man.iterrows():
    abridged = row['man_desc'].split('_')[0]
    at_int_man.loc[index, 'allele'] = abridged
at_int_man

In [ ]:
at_extein_cat_seqs = {}
for index, row in at_int_man.iterrows():
    if row['allele'] in at_extein_cat_seqs.keys():
           at_extein_cat_seqs.setdefault(row['allele'], []).append(row['target_name'])
    else:
        at_extein_cat_seqs[row['allele']] = [row['target_name']]
at_extein_cat_seqs

In [ ]:
at_uniq_targets ={}
for allele, targets in at_extein_cat_seqs.items():
    uniq_exts =  list(set(targets))
    at_uniq_targets[allele] = uniq_exts
for allele, targets in at_uniq_targets.items():
    for seq in targets:
        print('>' + seq + '\n' + at_extein_seqs[seq], file=open('../../atlit/manual/' + allele + '.ext', 'a'))

# Deep Lake

In [ ]:
dl30_int = pd.read_table('dl30.htab', sep='\t', lineterminator='\n', skiprows=0)
for indy,row in dl30_int.iterrows():
    if row['hmm _oord_from'] < 25 or row['hmm_coord_to'] >= 469:
        continue
    else:
        dl30_int.drop(index=indy, inplace=True)
dl30_int.reset_index(inplace=True, drop=True)
dl30_int

In [ ]:
for indy, row in dl30_int.iterrows():
    if row['hmm _oord_from'] == 1:
        continue
    if row['hmm_coord_to'] >= 469:
        if row['target_name'] == dl30_int.loc[(int(indy) - 1), 'target_name']:
            if row['hmm _oord_from'] > (dl30_int.loc[(int(indy) - 1), 'hmm_coord_to']):
                dl30_int.loc[(int(indy) - 1),
                            'env_coord_to'] = row['env_coord_to']

In [ ]:
dl_index_delete = []
for indy, row in dl30_int.iterrows():
    if row['hmm _oord_from'] == 1 and row['hmm_coord_to'] >= 469:
        continue
    if indy == 0:
        continue 
    if row['env_coord_to'] == dl30_int.loc[(int(indy) - 1), 'env_coord_to'] and row['target_name'] == dl30_int.loc[(int(indy) - 1), 'target_name']:
        dl_index_delete.append(indy)

In [ ]:
dl30_int.drop(index=dl_index_delete, inplace=True)
dl30_int.reset_index(inplace=True, drop=True)
dl30_int

In [ ]:
dl_faa_fasta = {}
with open("dl30.faa.eol") as dl_faa_file:
    for line in dl_faa_file:
        line = line.strip()
        if not line:
            continue
        if line.startswith(">"):
            active_sequence_name = line[1:].split(' ')[0]
            if active_sequence_name not in dl_faa_fasta.keys():
                dl_faa_fasta[active_sequence_name] = ""
            continue
        sequence = line
        dl_faa_fasta[active_sequence_name] = sequence

#dl_faa_fasta

In [ ]:
#dl_int_man.loc[10, 'env_coord_to'] = 217
dl_int_man.to_csv('../mix_dl_int_man.tab', index=None, sep='\t')

In [ ]:
dl_inteins = {}
for index, row in dl_int_man.iterrows():
    if row['target_name'] in dl_inteins.keys():
        dl_inteins.setdefault(row['target_name'], []).append(row['env_coord_to'])        
        
    else:
        dl_inteins[row['target_name']] = [ row['env_coord_to']]

In [ ]:
dl_inteins

In [ ]:
for prot in dl_inteins.keys():
    print('>' + prot + '\n' + dl_faa_fasta[prot], file=open('intein_cds.fasta', 'a'))

In [ ]:
for indy, row in dl_int_man.iterrows():
    start_site = (row['env_coord_from'] - 1)
    stop_site = (row['env_coord_to'])
    print(">" + row['target_name'] + '_' + str(row['dom#']) + '\n' +
          dl_faa_fasta[row['target_name']][start_site:stop_site], file=open('all_int_2.fasta', 'a') )

In [ ]:
dl_ext_sites = {}
for index, row in dl_int_man.iterrows():
    if row['target_name'] in dl_inteins.keys():
        dl_ext_sites.setdefault(row['target_name'], []).append(
            row['env_coord_from'])
        dl_ext_sites[row['target_name']].append(row['env_coord_to'])
    else:
        dl_ext_sites[row['target_name']] = [
            row['env_coord_from'], row['env_coord_to']]

dl_ext_sites

In [ ]:
dl_fix_ext_sites = {}
for target, vals in dl_ext_sites.items():
    dom_num = 0
    expression = ''
    while dom_num < len(vals):
        if dom_num == 0:
            expression = expression + ":" + str(vals[dom_num]) + ","
            dom_num += 1
        if dom_num % 2 != 0:
            expression = expression + (str(vals[dom_num] - 1)) + ":"
            dom_num += 1
        else:
            expression = expression + str(vals[dom_num]) + ','
            dom_num += 1
    dl_fix_ext_sites[target] = expression.split(',')
dl_fix_ext_sites

In [ ]:
# get concatenated extein sequences from fix_ext_sites
dl_extein_seqs = {}
for target, vals in dl_fix_ext_sites.items():
    sequence = ''
    for pair in vals:
        if pair[0] == ':':
            first_end = int(pair.split(':')[1])
            sequence = sequence + dl_faa_fasta[target][:first_end]
        else:
            start = int(pair.split(':')[0])

            if pair[-1] == ':':
                sequence = sequence + dl_faa_fasta[target][start:]
            else:
                end = int(pair.split(':')[1])
                sequence = sequence + dl_faa_fasta[target][start:end]

    dl_extein_seqs[target] = sequence

In [ ]:
for target,vals in dl_extein_seqs.items():
    print('>' + target + '\n' + vals, file=open('ext_seqs.fasta', 'a'))

In [ ]:
dl_ffn_fasta = {}
with open("dl30.ffn.eol") as dl_ffn_file:
    for line in dl_ffn_file:
        line = line.strip()
        if not line:
            continue
        if line.startswith(">"):
            active_sequence_name = line[1:].split(' ')[0]
            if active_sequence_name not in dl_ffn_fasta.keys():
                dl_ffn_fasta[active_sequence_name] = ""
            continue
        sequence = line
        dl_ffn_fasta[active_sequence_name] = sequence

dl_ffn_fasta

In [ ]:
dl_fix_ext_sites

In [ ]:
# get concatenated extein sequences from fix_ext_sites
dl_nt_extein_seqs = {}
for target, vals in dl_fix_ext_sites.items():
    sequence = ''
    for pair in vals:
        if pair[0] == ':':
            first_end = int(pair.split(':')[1])
            sequence = sequence + dl_ffn_fasta[target][:(first_end*3)]
        else:
            start = int(pair.split(':')[0])

            if pair[-1] == ':':
                sequence = sequence + dl_ffn_fasta[target][(start*3):]
            else:
                end = int(pair.split(':')[1])
                sequence = sequence + dl_ffn_fasta[target][(3*start):(3*end)]

    dl_nt_extein_seqs[target] = sequence
#for target, seqs in dl_nt_extein_seqs.items():
 #   print('>' + target + '_ext_full\n' +
  #        dl_ffn_fasta[target], file=open('nt_full_ext_seqs.fasta', 'a'))

In [ ]:
dl_nt_extein_seqs

In [ ]:
for target, seqs in dl_nt_extein_seqs.items():
    print('>' + target + '_ext\n' +
          seqs, file=open('nt_ext_seqs_2.fasta', 'a'))

In [ ]:
# all invidiual NT intein seqs
for indy, row in dl_int_man.iterrows():
    start_site = (row['env_coord_from'] - 1)
    stop_site = (row['env_coord_to'])
    print(">" + row['target_name'] + '_' + str(row['dom#']) + '\n' + dl_ffn_fasta[row['target_name']]
          [(start_site*3):(stop_site*3)], file=open('nt_int_2.fasta', 'a'))

In [ ]:
dl_int_man

In [ ]:
dl_full_inteins = []
for line in open('hensearch/dl30.blocked').readlines():
    line = line.split()
    dl_full_inteins.append(line[0])
dl_full_uniqs=  list(set(dl_full_inteins))

In [ ]:
dl30_int['int_size'] = ''
for indy, row in dl30_int.iterrows():
    if (row['target_name'] + '_' + str(row['dom#'])) in dl_full_uniqs:
        dl30_int.loc[indy, 'int_size'] = 'full'
    else:
        dl30_int.loc[indy, 'int_size'] = 'mini'
dl30_int        

In [ ]:
dl_multi_hen_dict = {}
for indy, row in dl30_int.iterrows():
    if row['target_name'] in dl_multi_hen_dict.keys():
        dl_multi_hen_dict.setdefault(
            row['target_name'], []).append(row['int_size'])

    else:
        dl_multi_hen_dict[row['target_name']] = [row['int_size']]

In [ ]:
# uri_multi int (later for mini inteins i think)
dl_ds_gene_count = len(dl_inteins)  # number of total genes with 1+ inteins
# multi_hens = 0 #skip but number of hens per multi genes
dl_multi_inteins = 0  # multiple inteins total (count of inteins)
# single intein in gene. if only 1 target_name (count of gene/inteins)
dl_single = 0
# single intein in gene w/HEN. (count of inteins) if only 1 target name and full
dl_single_hen = 0
# genes with multiple inteins (count of genes). if multiple target_names
dl_multiple_genes = 0
# multiple inteins with hen (count of inteins). if multiple target_names, and full
dl_multiple_hen = 0
# multiple_hen/multi_inteins = % of multiple inteins with HEN

for target, vals in dl_multi_hen_dict.items():
    if len(vals) == 1:
        dl_single += 1
        if vals[0] == 'full':
            dl_single_hen += 1
    if len(vals) > 1:

        dl_multiple_genes += 1
        dl_multi_inteins += len(vals)
        for size in vals:
            if size == 'full':
                dl_multiple_hen += 1
# need to change the flat values to be extensible
print('Deep Lake\nGenes with multiple inteins:\t' + str(dl_multiple_genes) + ' (' + str(dl_ds_gene_count) + ')\t' + str(dl_multiple_genes/dl_ds_gene_count) +
      '\nMulti-Inteins with HEN:\t\t' + str(dl_multiple_hen) + '\t\t' + str(dl_multiple_hen/dl_multi_inteins) +
      '\nSingle-Inteins with HEN:\t' + str(dl_single_hen) + '\t\t' + str(dl_single_hen/dl_single) + 
     '\nTotal full inteins:\t\t' + str(dl_single_hen + dl_multiple_hen) + ' (' + str(dl_single + dl_multi_inteins) + ')\t' + str((dl_single_hen+dl_multiple_hen)/(dl_single + dl_multi_inteins)))

In [ ]:
dl_nr_info = pd.read_csv('nr/dl_exteinbackground.tab', sep='\t', header=None, usecols=[0, 3])
dl_nr_info.columns = ['target_name', 'title']
dl_nr_info.drop_duplicates(subset='target_name', inplace=True)
dl_int_anno = dl30_int.merge(dl_nr_info, how='left', left_on='target_name', right_on='target_name')
dl_int_anno


In [ ]:
dl_int_anno['organism'] = dl_int_anno['title'].map(lambda x: re.search(r'\[.+?(?=\])|$', str(x)).group())
dl_int_anno['gene'] = dl_int_anno['title'].map(lambda x: re.search(r'^.+?(?=\[)|$', str(x)).group())
dl_int_anno.drop(['title'], axis=1, inplace=True)
dl_int_anno['organism'] = dl_int_anno['organism'].map(lambda x: x.lstrip('['))
dl_int_anno

## Deep Lake Frequencies

In [ ]:
for target, sites in dl_ext_sites.items():
    site = 0
    anchor = 0
    if len(sites) > 2:
        while site < (len(sites) -1):
        
            if site == 0:
                anchor = anchor +  (sites[site] *3)
                print(target + "_ext:" + str(anchor-3) + '-' + str(anchor+3) )
                site += 2
            if site % 2 == 0 and site != 0:
                anchor = anchor + ((sites[site] - sites[site-1]) *3) 
                print(target + "_ext:"+ str(anchor-3) + '-' + str(anchor+3))
                site +=2
    else:
        anchor = anchor +  (sites[site] *3)
        print(target + "_ext:" + str(anchor-3) + '-' + str(anchor+3) )


In [ ]:
dl_iis_sam = pd.read_table('../../DL30/hmmer/iis/shan/dl_final.sam',
                        usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13], header=None, sep='\t')
dl_iis_sam.columns = ['read', 'bits', 'target_name',
                   'pos', 'mapq', 'cigar', 'mate_target', 'mpos', 'tlen' ,  'nm', 'as', 'xs']
dl_iis_sam

In [ ]:
dl_iis_counter = {}
dl_iis_lens = {}
#number  = 0
for index, row in dl_iis_sam.iterrows():
    if '3D' in row['cigar']:
        
        if row['target_name'][:-4] in dl_iis_counter.keys():
            dl_iis_counter.setdefault(row['target_name'][:-4], []).append(row['cigar'])
        else:
            dl_iis_counter[row['target_name'][:-4]] = [row['cigar']]
for target, vals in dl_iis_counter.items():
    dl_iis_lens[target] = len(vals)
dl_iis_lens

# Outputs and Plots

In [ ]:
at22_int.description_of_target.unique()

In [ ]:
at_int_anno[at_int_anno['description_of_target'] == 'Bacteriophage T4-like capsid assembly protein (Gp20)']